In [22]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


In [23]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [24]:
# Drop Null values of Target 
df = model_data.dropna(subset=['90_in_24mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [25]:
#Split the time interval for model building and Validation of oot
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]
print('********* Data Splitted *********')
print(model_df1.shape, oot_df1.shape)

********* Data Splitted *********
(39010, 2831) (12128, 2831)


In [26]:
features_list =[
    
#Bureau Intutive features
'Num_Enq_all_loans_last_9M_3M',
'Utilization_all',
'total_emi',
'Utilization_CREDIT CARD_opened_last_36M',
'Utilization_PERSONAL LOAN_opened_last_9M',
'num_CREDIT CARD_enq_last_6M',
'num_CONSUMER LOAN_enq_last_36M',
'Num_of_PERSONAL LOAN_opened_last6M_12M',
'highest_sanction_amount_credit_cards',
'num_CONSUMER LOAN_enq_last_6M',
'num_PL_LE_1_Lac_opened_in_last_12M',
'Total_Sanction_amt_PERSONAL LOAN_Opened_last_24M',
'Age_of_oldest_open_account',
    
#Banking Intutive features

'num_credit_gt_salary',    
'upi_trans_count',
'ratio_balance_after_7_days_max_credit_M3',    
'ratio_max_credit_to_total_credit_M2',
'spend_vs_salary_ratio',
'bounced_transactions_count',
'ratio_total_debit_to_max_credit_M2_M3',
'min_balance_last_3_months',
'min_balance_M2',
'days_balance_lt_5000_M3',
'days_balance_lt_10000_M3',
'Max_Credit_Avg_M1_M2_M3',
'Max_Credit_Diff_M1_M2_M3',
'amount_debits_last_2_months',
'sum_insurance_debit',
'avg_num_debits_last_2_months',
'upi_trans_debit_sum',
'cash_withdrawal_sum',
    
#new banking features
'diff_btw_stated_salary_vs_actual_salary',
'balance_6th_day_M2',
'ratio_6th_day_balance_to_max_credit_M3',

#Loan application Features

'deviation_from_median_income/standard_deviation_income#last_365_Days',
'loan_amount_to_income#last_365_Days',



#PG synthetic Features
'distinct_merchant',
'MEAN(raw.amount WHERE device_os_category = Windows)#ever',
'days_on_pg',
'NUM_UNIQUE(pg_success_raw_data.merchant_id WHERE revised_merchant_category = marketplace)',
'amt_first_android_txn', 
    
#ID Features
'dm206s',
'g406s',
'balmag01',

# # Reconcilated Features
'percentage_ever_delinquent', 
'months_since_most_recent_delinquency_installment', 
'percentage_utilization_gt_75',
'Num_of_installment_loan_opened_last_6M',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
'month_year',
'90_in_24mob'
              
]




In [27]:

model_df = model_df1[features_list]
oot_df = oot_df1[features_list]

# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '90_in_24mob']) # Adjust 'target' to your actual target column name
y_model = model_df['90_in_24mob']

X_oot = oot_df.drop(columns=['month_year', '90_in_24mob'])
y_oot = oot_df['90_in_24mob']

#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)



/tmp/ipykernel_3022882/282645959.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3022882/282645959.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [29]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((23406, 50), (15604, 50), (23406,), (15604,))

In [30]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(18724, 50) (4682, 50) (15604, 50)


In [31]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)

In [32]:

params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
    'max_depth': 2,
    'subsample': 0.3,
    'colsample_bytree': 0.3,
    'lambda': 12.6,
    'alpha': 4.8,
    'gamma': 5.6,
    'random_state': 24   
}

evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 500,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.60528	eval-auc:0.59034
[10]	train-auc:0.66049	eval-auc:0.65266
[20]	train-auc:0.66989	eval-auc:0.66034
[30]	train-auc:0.67141	eval-auc:0.66393
[40]	train-auc:0.67272	eval-auc:0.66546
[50]	train-auc:0.67456	eval-auc:0.66688
[60]	train-auc:0.67551	eval-auc:0.66869
[70]	train-auc:0.67560	eval-auc:0.66844
[80]	train-auc:0.67658	eval-auc:0.66889
[90]	train-auc:0.67645	eval-auc:0.66964
[100]	train-auc:0.67749	eval-auc:0.67070
[110]	train-auc:0.67816	eval-auc:0.67061
[120]	train-auc:0.67930	eval-auc:0.67099
[130]	train-auc:0.68035	eval-auc:0.67228
[140]	train-auc:0.68172	eval-auc:0.67358
[150]	train-auc:0.68262	eval-auc:0.67448
[160]	train-auc:0.68336	eval-auc:0.67536
[170]	train-auc:0.68379	eval-auc:0.67563
[180]	train-auc:0.68466	eval-auc:0.67614
[190]	train-auc:0.68531	eval-auc:0.67639
[200]	train-auc:0.68564	eval-auc:0.67663
[210]	train-auc:0.68622	eval-auc:0.67711
[220]	train-auc:0.68638	eval-auc:0.67732
[230]	train-auc:0.68689	eval-auc:0.67737
[240]	train-auc:0.68711	eva

In [33]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.70, Gini: 0.39, KS: 0.28
Eval AUC: 0.68, Gini: 0.37, KS: 0.27
Test AUC: 0.67, Gini: 0.35, KS: 0.25
OOT AUC: 0.69, Gini: 0.38, KS: 0.28


In [34]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.410571         0.333991        0.554306        0.271553   1873   
9          0.259615         0.242253        0.271416        0.219082   1872   
8          0.215278         0.202364        0.219071        0.187991   1872   
7          0.177256         0.176652        0.187944        0.166784   1873   
6          0.153312         0.158372        0.166728        0.150484   1872   
5          0.137821         0.143221        0.150483        0.135417   1872   
4          0.115857         0.126918        0.135411        0.119120   1873   
3          0.098291         0.111655        0.119118        0.104576   1872   
2          0.079594         0.098392        0.104573        0.091853   1872   
1          0.034704         0.083680        0.091849        0.066877   1873   

        label_good  label_bad  
Decile                         
10            1104        769  
9             1386        486  
8             1469        403  
7             1541        332  
6             1585        287  
5             1614        258  
4             1656        217  
3             1688        184  
2             1723        149  
1             1808         65

In [35]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.381807         0.332520        0.538680        0.270683   1561   
9          0.274359         0.242521        0.270639        0.219051   1560   
8          0.209615         0.201809        0.219033        0.187097   1560   
7          0.174888         0.176654        0.187065        0.166425   1561   
6          0.153846         0.157612        0.166421        0.149516   1560   
5          0.134615         0.141896        0.149513        0.134312   1560   
4          0.122998         0.126247        0.134304        0.118363   1561   
3          0.109615         0.111316        0.118339        0.104454   1560   
2          0.084615         0.098129        0.104441        0.091639   1560   
1          0.062780         0.083425        0.091638        0.065649   1561   

        label_good  label_bad  
Decile                         
10             965        596  
9             1132        428  
8             1233        327  
7             1288        273  
6             1320        240  
5             1350        210  
4             1369        192  
3             1389        171  
2             1428        132  
1             1463         98

In [36]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.393240         0.326254        0.536231        0.267440   1213   
9          0.272053         0.239909        0.267371        0.217846   1213   
8          0.228359         0.201589        0.217839        0.186854   1213   
7          0.193894         0.176234        0.186853        0.166654   1212   
6          0.159110         0.158695        0.166625        0.151724   1213   
5          0.122836         0.144444        0.151697        0.136972   1213   
4          0.114686         0.129288        0.136924        0.121458   1212   
3          0.104699         0.113531        0.121450        0.106370   1213   
2          0.089860         0.099728        0.106369        0.093436   1213   
1          0.043693         0.085126        0.093433        0.066871   1213   

        label_good  label_bad  
Decile                         
10             736        477  
9              883        330  
8              936        277  
7              977        235  
6             1020        193  
5             1064        149  
4             1073        139  
3             1086        127  
2             1104        109  
1             1160         53

In [37]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
35                                         days_on_pg       133.0
39                                              g406s       120.0
0                        Num_Enq_all_loans_last_9M_3M        71.0
46                          Num_Enq_non_BANK_last_12M        64.0
4            Utilization_PERSONAL LOAN_opened_last_9M        55.0
33                                  distinct_merchant        54.0
45                           No of enq in past 90days        54.0
37                              amt_first_android_txn        52.0
34  MEAN(raw.amount WHERE device_os_category = Win...        49.0
12                         Age_of_oldest_open_account        48.0
1                                     Utilization_all        47.0
18                         bounced_transactions_count        40.0
8                highest_sanction_amount_credit_cards        38.0
9                       num_CONSUMER LOAN_enq_last_6M        36.0
40        

In [38]:
feature_importance_df

Feature  Importance
35                                         days_on_pg       133.0
39                                              g406s       120.0
0                        Num_Enq_all_loans_last_9M_3M        71.0
46                          Num_Enq_non_BANK_last_12M        64.0
4            Utilization_PERSONAL LOAN_opened_last_9M        55.0
33                                  distinct_merchant        54.0
45                           No of enq in past 90days        54.0
37                              amt_first_android_txn        52.0
34  MEAN(raw.amount WHERE device_os_category = Win...        49.0
12                         Age_of_oldest_open_account        48.0
1                                     Utilization_all        47.0
18                         bounced_transactions_count        40.0
8                highest_sanction_amount_credit_cards        38.0
9                       num_CONSUMER LOAN_enq_last_6M        36.0
40                                           balmag01        35.0
7              Num_of_PERSONAL LOAN_opened_last6M_12M        34.0
36  NUM_UNIQUE(pg_success_raw_data.merchant_id WHE...        30.0
41                         percentage_ever_delinquent        28.0
14                                    upi_trans_count        28.0
10                 num_PL_LE_1_Lac_opened_in_last_12M        27.0
5                         num_CREDIT CARD_enq_last_6M        25.0
44             Num_of_installment_loan_opened_last_6M        24.0
31  deviation_from_median_income/standard_deviatio...        21.0
27                                cash_withdrawal_sum        20.0
2                                           total_emi        17.0
3             Utilization_CREDIT CARD_opened_last_36M        17.0
6                      num_CONSUMER LOAN_enq_last_36M        17.0
22                           days_balance_lt_10000_M3        15.0
43                       percentage_utilization_gt_75        15.0
42   months_since_most_recent_delinquency_installment        14.0
21                            days_balance_lt_5000_M3        12.0
20                                     min_balance_M2         7.0
11   Total_Sanction_amt_PERSONAL LOAN_Opened_last_24M         7.0
25                       avg_num_debits_last_2_months         7.0
13                               num_credit_gt_salary         6.0
38                                             dm206s         6.0
32                loan_amount_to_income#last_365_Days         6.0
29                                 balance_6th_day_M2         4.0
19                          min_balance_last_3_months         3.0
26                                upi_trans_debit_sum         3.0
23                            Max_Credit_Avg_M1_M2_M3         3.0
28            diff_btw_stated_salary_vs_actual_salary         3.0
15           ratio_balance_after_7_days_max_credit_M3         2.0
30             ratio_6th_day_balance_to_max_credit_M3         2.0
24                                sum_insurance_debit         1.0
17                              spend_vs_salary_ratio         1.0
16                ratio_max_credit_to_total_credit_M2         1.0

In [39]:
#Save the Pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_BBILAP_Train_Pred_9024.csv',index=False)
Eval.to_csv('LeoPayu_BBILAP_Eval_Pred_9024.csv',index=False)
Test.to_csv('LeoPayu_BBILAP_Test_Pred_9024.csv',index=False)
Oot.to_csv('LeoPayu_BBILAP_Oot_Pred_9024.csv',index=False)


In [40]:
#Save the model
xgb_model.save_model('LeoPayu_BBILAP_90_in_24mob.json')
print('Model Saved')

Model Saved
